In [17]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS shir_hashes (
    ID INT,
    TimestampColumn TIMESTAMP,
    HashValue STRING,
    isDone BOOLEAN
)
"""


spark.sql(create_table_sql)

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 19, Finished, Available, Finished)

DataFrame[]

In [20]:
spark.sql("SELECT * FROM shir_hashes ORDER BY ID DESC").show()

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 22, Finished, Available, Finished)

+---+---------------+---------+------+
| ID|TimestampColumn|HashValue|isDone|
+---+---------------+---------+------+
+---+---------------+---------+------+



In [19]:
spark.sql(f"delete from shir_hashes")

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 21, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint]

In [21]:
max_id_row = spark.sql(f"SELECT MAX(ID) AS max_id FROM shir_hashes").first()

if max_id_row.max_id == None: #only needed when control table is truncated
    max_id = 0
    currentFileHash ='ASD'
    new_id = 1
else:
    max_id = max_id_row.max_id
    new_id = max_id + 1
    #Get current HASH
    getHASH = spark.sql(f"SELECT HashValue AS currentHASH FROM shir_hashes WHERE ID = {max_id}").first()
    currentFileHash = getHASH.currentHASH
    print(getHASH.currentHASH)
 

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 23, Finished, Available, Finished)

In [22]:
import hashlib

def hash_file(file_path):
    # Create a hash object
    hasher = hashlib.sha256()

    # Open the file in binary mode
    with open(file_path, 'rb') as f:
        # Read the file in chunks
        for chunk in iter(lambda: f.read(4096), b''):
            # Update the hash object with the chunk
            hasher.update(chunk)

    # Get the hexadecimal representation of the hash
    file_hash = hasher.hexdigest()

    return file_hash


StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 24, Finished, Available, Finished)

In [27]:
import os

directory_path = "/lakehouse/default/Files/SHIR"

if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created.")
else:
    print(f"Directory '{directory_path}' already exists.")

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 29, Finished, Available, Finished)

Directory '/lakehouse/default/Files/SHIR' already exists.


In [28]:
import urllib.request

url = "https://download.microsoft.com/download/E/4/7/E4771905-1079-445B-8BF9-8A1A075D8A10/Release%20Notes.doc"
filename = "/lakehouse/default/Files/SHIR/Release_Notes.doc"

urllib.request.urlretrieve(url, filename)

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 30, Finished, Available, Finished)

('/lakehouse/default/Files/SHIR/Release_Notes.doc',
 <http.client.HTTPMessage at 0x7297c127b910>)

In [29]:
#currentFile = '/lakehouse/default/Files/SHIR/currentFile/Release_Notes.doc'
#currentFileHash = hash_file(currentFile)

newFileHash = hash_file(filename)

# Compare hashes
if currentFileHash == newFileHash:
    print("All files have the same hash.")
    toRunOrNotToRun = 'noRun'
else:
    print("Files have different hashes.")
    toRunOrNotToRun = 'toRun'

    insert_query = f"INSERT INTO shir_hashes VALUES ({new_id}, CURRENT_TIMESTAMP(), '{newFileHash}',False)"
    spark.sql(insert_query)


print(toRunOrNotToRun)

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 31, Finished, Available, Finished)

Files have different hashes.
toRun


In [30]:
mssparkutils.notebook.exit(toRunOrNotToRun)

StatementMeta(, 7347ed7f-6949-4a00-8a5f-18e02e9b8014, 32, Finished, Available, Finished)

ExitValue: toRun